In [ ]:
from linkedin_scraper.linkedin_scraper import actions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import pandas as pd

import gspread
import df2gspread as d2g
from df2gspread import df2gspread as d2g


from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
driver = webdriver.Chrome()

email = "LINKEDIN IN USERNAME"
password = "LINKEDIN PWD"
actions.login(driver, email, password)


In [ ]:
# SPECIFY RELEVANT INPUT HERE
# FILENAME: WHERE TO SAVE RESULTS TO
# PG: LIST OF THE # OF PAGES IN LINKEDIN, RESPECTIVE TO EACH OF THE LINKS IN "S"
# S: LIST OF LINKEDIN LINKS, OF COMPANIES, TO SCRAPE

filename = 'Digitalization LinkedIn'
pg = [3, 1, 2]

s = '''https://www.linkedin.com/search/results/companies/?companyHqGeo=%5B%22106232576%22%2C%2290009712%22%5D&industryCompanyVertical=%5B%221855%22%2C%223105%22%2C%223104%22%2C%223103%22%2C%223107%22%2C%2296%22%2C%226%22%2C%222458%22%2C%223106%22%2C%223130%22%2C%224%22%2C%223244%22%5D&keywords=energy%20efficiency&origin=FACETED_SEARCH&searchId=cef66829-37e7-4038-b59a-01bb8ee486b5&sid=zUb
https://www.linkedin.com/search/results/companies/?companyHqGeo=%5B%22106232576%22%2C%2290009712%22%5D&industryCompanyVertical=%5B%221855%22%2C%223105%22%2C%223104%22%2C%223103%22%2C%223107%22%2C%2296%22%2C%226%22%2C%222458%22%2C%223106%22%2C%223130%22%2C%224%22%2C%223244%22%5D&keywords=waste%20reduce&origin=GLOBAL_SEARCH_HEADER&sid=K(R
https://www.linkedin.com/search/results/companies/?companyHqGeo=%5B%22106232576%22%2C%2290009712%22%5D&industryCompanyVertical=%5B%221855%22%2C%223105%22%2C%223104%22%2C%223103%22%2C%223107%22%2C%2296%22%2C%226%22%2C%222458%22%2C%223106%22%2C%223130%22%2C%224%22%2C%223244%22%5D&keywords=energy%20optimization&origin=GLOBAL_SEARCH_HEADER&sid=U(w'''

In [ ]:
urls = s.split('\n')
list_zip = zip(urls, pg)
urls = list(list_zip)

lnks = []

In [ ]:
import time

for url in urls:
    driver.get(url[0])
    time.sleep(2)
    for i in range(0, url[1]):
        driver.execute_script("window.scrollTo(0, 50);")
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'entity-result__item')))
        cards = driver.find_elements(By.CLASS_NAME, 'entity-result__item')

        for card in cards:
            name = card.find_element(By.CLASS_NAME, "entity-result__title-text").text
            lnk = card.find_element(By.CLASS_NAME, "entity-result__title-text").find_element(By.TAG_NAME, "a").get_attribute('href')
            lnks.append([name, lnk])                        
        if i < (url[1]-1):
            while True:
                try: 
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'artdeco-pagination')))
                    pages = driver.find_element(By.CLASS_NAME, "artdeco-pagination")

                    WebDriverWait(pages, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'artdeco-pagination__button--next'))).click()
                    break
                except:
                    pass
    
    

In [ ]:
df = pd.DataFrame(lnks, columns=['name', 'url'])

In [ ]:
all_companies = []

In [ ]:
# RETRIEVING ADDITIONAL INFORMATION FROM LINKEDIN

for link in lnks[64:]:
    driver.get(link)
    time.sleep(5)

    name, tagline, domain, city, followers, website, phone, industry, size, founded, speciality = [None] * 11

    try:
        name = driver.find_element(By.CLASS_NAME, 'org-top-card-summary__title').text
    except:
        pass
    
    try:
        tagline = driver.find_element(By.CLASS_NAME, 'org-top-card-summary__tagline').text
    except:
        pass

    try: 
        domain = driver.find_elements(By.CLASS_NAME, 'org-top-card-summary-info-list__info-item')[0].text
        city = driver.find_elements(By.CLASS_NAME, 'org-top-card-summary-info-list__info-item')[1].text 
        followers = driver.find_elements(By.CLASS_NAME, 'org-top-card-summary-info-list__info-item')[2].text
    except:
        pass
    
    company = {
            'name' : name if name is not None else None,
            'tagline' : tagline if tagline is not None else None,
            'domain' : domain if domain is not None else None,
            'city' : city if city is not None else None,
            'followers' : followers if followers is not None else None,
            'linkedin_link' : link
    }

    try: 
        about_index = [i.text for i in driver.find_elements(By.CLASS_NAME, 'org-page-navigation__item')].index("About")
        about_link = driver.find_elements(By.CLASS_NAME, 'org-page-navigation__item')[about_index].find_element(By.TAG_NAME, "a").get_attribute('href')

        # ABOUT PAGE

        driver.get(about_link)
        time.sleep(5)

        about_card = driver.find_element(By.TAG_NAME, 'dl')

        if len(about_card.find_elements(By.TAG_NAME, 'dd')) == len(about_card.find_elements(By.TAG_NAME, 'dt')):    
            params = [list(a) for a in zip([i.text for i in about_card.find_elements(By.TAG_NAME, 'dt')], [i.text for i in about_card.find_elements(By.TAG_NAME, 'dd')])]
            for c in params: 
                company[c[0]] = c[1]

        else:
            for i, dt in enumerate(about_card.find_elements(By.TAG_NAME, 'dt')):
                dt = about_card.find_elements(By.TAG_NAME, 'dt')[i]
                if dt.text == "Website":
                    company[dt.text] = about_card.find_elements(By.TAG_NAME, 'dd')[i].text
                if dt.text == "Phone":
                    company[dt.text] = about_card.find_elements(By.TAG_NAME, 'dd')[i].text
                if dt.text == "Industry":
                    company[dt.text] = about_card.find_elements(By.TAG_NAME, 'dd')[i].text
                if dt.text == "Company size":
                    company[dt.text] = about_card.find_elements(By.TAG_NAME, 'dd')[i].text
                if dt.text == "Founded":
                    company[dt.text] = about_card.find_elements(By.TAG_NAME, 'dd')[i+1].text
                if dt.text == "Specialties":
                    company[dt.text] = about_card.find_elements(By.TAG_NAME, 'dd')[i+1].text

    except: 
        pass
    
    all_companies.append(company)

In [ ]:
df_re = pd.DataFrame(all_companies)

In [ ]:
df_re.head(3)

In [ ]:
# YOU NEED TO MAKE SURE ALL REQUIREMENTS FOR df2gspread ARE SATIFIED

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'PATH TO jsonFileFromGoogle.json', scope)
gc = gspread.authorize(credentials)



spreadsheet_key = 'GOOGLE SHEETS ID'
wks_name = filename
d2g.upload(df_re.reset_index(), spreadsheet_key, wks_name, credentials=credentials, row_names=True)


In [ ]:
driver.quit()